In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import string
import re
import json


with open("scraper.json", mode="r") as f:
    example_arr = json.load(f)

example_arr


 database access control levels\nDevelop routines for end users to facilitate best practices database use\nProvide position-related subject matter guidance to members of the Applications team, and the Information Technology Division team as appropriate\nOccasional travel to various work locations is required to fulfill the duties of the position\nPerform other related duties as assigned\n\nEducation and skills required:\nPost-secondary education in Computer Science or a related field\nCertification in Database Management and Administration\nMinimum 5 years of related progressive experience with designing, building, installing, configuring and supporting database servers, including Microsoft SQL, Access, MySQL\nStrong understanding of database structures, theories, principles, and practices\nHands-on database tuning and troubleshooting experience\nExperience with data processing flowcharting techniques\nProject management experience\nGood understanding of organizational goals and object

### We will do the following:
1. Figure out number of columns.
2. Append every row.
3. Be done!


In [268]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def compare_two_synsets(existing_word, word_to_add):
    '''
        if words are not similar, create a new one_hot_encoding,
        if words are too similar, keep the existing word as the encoding.
    '''
    prev_existing_word = existing_word
    prev_word_to_add = word_to_add

    try:
        words_1 = wn.synsets(existing_word)
        words_2 = wn.synsets(word_to_add)
        
        for word_1 in words_1:
            for word_2 in words_2:
                similarity_score = word_1.path_similarity(word_2)
                
                ## Non-None check and similarity check
                if similarity_score and similarity_score >= 0.5:
                    return prev_existing_word 
        
        return prev_word_to_add

    except KeyError:     ## We let this go
        pass

    return prev_word_to_add

def find_nth(string, substring, n):
   if (n == 1):
       return string.find(substring)
   else:
       return string.find(substring, find_nth(string, substring, n - 1) + 1)

In [269]:
from nltk.stem import WordNetLemmatizer
ps = WordNetLemmatizer()


used_words = [] ## old_encodings that we have used before
new_words = [] ## new_encodings for the new column
stop_words = set(stopwords.words("english"))

## Figure out one-hot encodings for title
final_data = []
for title, description in example_arr:
    new_row = []

    new_row.append(title.lower().strip()) ## job_title
    new_row.append(description[0:min(find_nth(description, ".", 2), find_nth(description, "\n", 6)) + 1].lower()) ## Short Description

    description = re.split('[\#\$\%\&\'\ \.\/\:\;\<\=\>\?\@\[\]\^\`\{\}\|\,|\\n]+', description) ## Split whitespace and punctuation

    ## Remove stop_words, then stem leftover words
    new_row.append(",".join([ps.lemmatize(word.lower()) for word in description if word not in stop_words and word != '' and not RepresentsInt(word)]))
    nltk_name = []
    for new_word in re.split('[\W+]', title):
        result = new_word ## default case when there are no used words. We assign new column = 1 by default
        for old_word in used_words:
            result = compare_two_synsets(old_word, new_word)
            if result == new_word:
                pass 
            else:
                print("%s and %s are similar" % (old_word, new_word))
                nltk_name.append(result) ## similarity > 0.34. Assign 1 to existing encoding.
                break
            
        if result == new_word: ## similarity < 0.34 compared to all used words
            nltk_name.append(result)
    
    new_row.append((' '.join(nltk_name)).lower()) ## nltk_title
    final_data.append(new_row)

print(final_data[0])
# from nltk.stem import PorterStemmer 
# ps = PorterStemmer() 


# final_data = []
# ## Figure out one-hot encodings for title
# for title, description in example_arr:
#     new_row = [0 for i in range(len(empty_frame.columns))]

#     new_row[columns_indices["Job_Title"]] = title
#     new_row[columns_indices["Short_Desc"]] = description[0:100]

#     description = re.split('\W+', description) ## Split whitespace and punctuation

#     ## Remove stop_words, then stem leftover words
#     new_description = [word for word in description if word not in stop_words]
#     new_row[columns_indices["Skills"]] = ",".join(new_description)


#     for new_word in re.split('\W+', title):
#         ## We want to treat Software like software_engineer
#         new_word = new_word if new_word != "Software" else "software_engineer"
#         if new_word in columns_indices:
#             new_row[columns_indices[new_word]] = 1

    
#     final_data.append(new_row)


# final_data


    
    
    
    
  

    
    
    
    
  

['senior oracle database administrator', 'posting id:\n20152\n\nposition type:\nregular/full-time\n\n', 'posting,id,position,type,regular,full-time,city,vancouver,bc,canada,location,vancouver,h,o,-,canfor,cwpm_1000,(bc13),canfor’s,strength,come,diversity,skilled,resilient,workforce,a,company,continues,grow,diversify,career,opportunity,let,u,take,opportunity,tell,could,best,fit,canfor,the,opportunity,the,senior,oracle,database,administrator,(dba),responsible,aspect,corporate,database,strategy,planning,security,implementation,consulting,technical,support,the,dba,evaluates,selects,implement,database,software,utility,hardware,meet,corporate,business,system,requirement,according,principle,policy,information,technology,architecture,defined,senior,management,the,goal,database,administrator,develop,efficient,effective,facility,procedure,design,storage,maintenance,protection,corporate,data,resource,the,position,generally,involved,number,project,may,require,work,outside,regular,business,hour,thi

In [270]:
final_data[0]

['senior oracle database administrator',
 'posting id:\n20152\n\nposition type:\nregular/full-time\n\n',
 'posting,id,position,type,regular,full-time,city,vancouver,bc,canada,location,vancouver,h,o,-,canfor,cwpm_1000,(bc13),canfor’s,strength,come,diversity,skilled,resilient,workforce,a,company,continues,grow,diversify,career,opportunity,let,u,take,opportunity,tell,could,best,fit,canfor,the,opportunity,the,senior,oracle,database,administrator,(dba),responsible,aspect,corporate,database,strategy,planning,security,implementation,consulting,technical,support,the,dba,evaluates,selects,implement,database,software,utility,hardware,meet,corporate,business,system,requirement,according,principle,policy,information,technology,architecture,defined,senior,management,the,goal,database,administrator,develop,efficient,effective,facility,procedure,design,storage,maintenance,protection,corporate,data,resource,the,position,generally,involved,number,project,may,require,work,outside,regular,business,hour,t

In [271]:
processed_df = pd.DataFrame(final_data, columns=["Job_Title", "Short_Desc", "Skills", "Predicted_Job_Title"])

In [272]:
processed_df.columns = [column.lower() for column in processed_df.columns.to_list()]


In [273]:
processed_df.head()

,job_title,short_desc,skills,predicted_job_title
0,senior oracle database administrator,posting id:\n20152\n\nposition type:\nregular/...,"posting,id,position,type,regular,full-time,cit...",senior oracle database administrator
1,database administrator (ms sql server),seeking a database administrator (12 month con...,"seeking,database,administrator,(12,month,contr...",database administrator ms sql server
2,database administrator,a service-driven paper and packaging products ...,"a,service-driven,paper,packaging,product,compa...",database administrator
3,sql database administrator,"we are traffic tech, the hardest working team ...","we,traffic,tech,hardest,working,team,transport...",sql database administrator
4,database administrator (dba),,"specific,skill,collect,document,user,requireme...",database administrator dba


In [274]:
processed_df.iloc[0]["skills"]

'posting,id,position,type,regular,full-time,city,vancouver,bc,canada,location,vancouver,h,o,-,canfor,cwpm_1000,(bc13),canfor’s,strength,come,diversity,skilled,resilient,workforce,a,company,continues,grow,diversify,career,opportunity,let,u,take,opportunity,tell,could,best,fit,canfor,the,opportunity,the,senior,oracle,database,administrator,(dba),responsible,aspect,corporate,database,strategy,planning,security,implementation,consulting,technical,support,the,dba,evaluates,selects,implement,database,software,utility,hardware,meet,corporate,business,system,requirement,according,principle,policy,information,technology,architecture,defined,senior,management,the,goal,database,administrator,develop,efficient,effective,facility,procedure,design,storage,maintenance,protection,corporate,data,resource,the,position,generally,involved,number,project,may,require,work,outside,regular,business,hour,this,position,also,participate,on-call,rotation,it,operation,support,travel,may,involved,this,role,based,ei

In [275]:
processed_df.columns

Index(['job_title', 'short_desc', 'skills', 'predicted_job_title'], dtype='object')

In [276]:
prev_df = pd.read_csv("processScraper.csv", index_col=0)

In [277]:
final_df = pd.concat([prev_df, processed_df])

In [ ]:
final_df.index = [i for i in range(final_df.shape[0])]

In [278]:
final_df.to_csv("processScraper.csv")

In [279]:
final_df[["skills", "job_title"]].to_csv("processScraper_wo_encoding.csv")

In [280]:
final_df

,job_title,short_desc,skills,predicted_job_title
0,jr. sous chef,golden eagle golf club is a 36 hole golf facil...,"golden,eagle,golf,club,hole,golf,facility,situ...",jr sous chef
1,executive head chef,executive head chef\njob description\n,"executive,head,chef,job,description,position,e...",executive head chef
2,sous chef,north shore catering is looking for chef and s...,"north,shore,catering,looking,chef,sou,chef,pos...",sous chef
3,looking to build a new team of line cooks,urgently hiring\nworking chef and sous chef an...,"urgently,hiring,working,chef,sou,chef,line,coo...",looking to build a new team of line cooks
4,chef de train,"pour faire carrière au chemin de fer, il est e...","pour,faire,carrière,au,chemin,de,fer,il,est,es...",chef de train
...,...,...,...,...
40,oracle database enterprise edition administrator,samiti technology company is actively seeking ...,"samiti,technology,company,actively,seeking,ora...",oracle database enterprise edition administrator
41,database administrator,"grow with the best. join a smart, creative, an...","grow,best,join,smart,creative,inspired,team,wo...",database administrator
42,database administrator,"at caesars windsor, it’s our mission to inspir...","at,caesar,windsor,it’s,mission,inspire,grown-u...",database administrator
43,database administrator,"salary: $32.00/hourly\njob type: full time, pe...","salary,hourly,job,type,full,time,permanent,sta...",database administrator
